In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm

In [35]:
data_response_1 = pd.read_csv('aug_to_oct/plogs.csv')
data_response_2 = pd.read_csv('feb_to_apr/plogs.csv')
# data_response_3 = pd.read_csv('may_to_june/plogs.csv')
# data_response_4 = pd.read_csv('nov_to_jan/plogs.csv')

q_data_1 = pd.read_csv('aug_to_oct/pdets.csv')[['problem_id','skills']]
q_data_2 = pd.read_csv('feb_to_apr/pdets.csv')[['problem_id','skills']]
# q_data_3 = pd.read_csv('may_to_june/pdets.csv')[['problem_id','skills']]
# q_data_4 = pd.read_csv('nov_to_jan/pdets.csv')[['problem_id','skills']]


In [7]:
data_response = pd.concat([data_response_1,data_response_2])
q_data = pd.merge(q_data_1, q_data_2, how = 'outer', on = ['problem_id', 'skills'])

In [59]:
data_response.head()

,log_id,student_id,assignment_id,problem_id,start_time,time_on_task,answer_before_tutoring,fraction_of_hints_used,attempt_count,answer_given,problem_completed,correct
0,3981410,402701,364445,1456214,2020-08-01 02:23:26.565000-04:00,11.450,True,0.0,1,False,True,True
1,3981410,402701,364445,1558472,2020-08-01 02:23:47.681000-04:00,11.818,True,NaN,1,False,True,True
2,3981410,402701,364445,1558473,2020-08-01 02:24:07.132000-04:00,9.738,True,NaN,1,False,True,True
3,3981410,402701,364445,1558474,2020-08-01 02:24:19.276000-04:00,10.741,True,NaN,1,False,True,True
4,3981410,402701,364445,1456221,2020-08-01 02:24:31.972000-04:00,NaN,NaN,NaN,0,False,False,NaN


In [60]:
q_data.head()

,problem_id,skills
0,33.0,['8.NS.A.2-1']
1,35.0,['8.NS.A.2-1']
2,37.0,['8.NS.A.2-1']
3,39.0,['8.NS.A.2-1']
4,58.0,NaN


In [61]:
data_response = data_response.dropna(subset=['correct'])
q_data = q_data.dropna(subset=['skills'])
data_response = data_response.loc[data_response['problem_id'].isin(q_data['problem_id'].unique())]

In [65]:
response_data = dict()
stu_response_data = dict() 
stu_num = np.random.choice(np.arange(len(data_response['student_id'].unique())), size=1000, replace=False)
least_respone_num = 80
original_stu_map = dict()
original_cnt_stu = 0

for stu in data_response["student_id"].unique():
    original_stu_map[original_cnt_stu] = stu
    original_cnt_stu += 1

In [66]:
for stu in tqdm(stu_num, desc='Filter student'):
    stu_data = data_response.loc[data_response["student_id"] == original_stu_map[stu]]
    for data in stu_data.values:
        tmp_data = (stu, data[3])
        response_data[tmp_data] = (data[11] == True)

for key, value in response_data.items():
    if key[0] not in stu_response_data:
        stu_response_data[key[0]] = []
    stu_response_data[key[0]].append([int(key[0]), key[1], int(value)])

Filter student: 100%|██████████| 1000/1000 [00:10<00:00, 93.04it/s]


In [68]:
stu_map = dict()
cnt_stu = 0
question_set = set()
cnt_question = 0
question_map = dict()
concept_set = set()
cnt_concept = 0
concept_map = dict()


for key, value in tqdm(stu_response_data.items(), desc='Remap student_id, question_id and concept_id'):
    if len(value) >= least_respone_num:
        stu_map[key] = cnt_stu
        cnt_stu += 1
        for data in value:
            question_set.add(data[1])
            for concept in q_data.loc[q_data["problem_id"] == data[1]]['skills'].iloc[0].split(','):
                concept_set.add(concept)


for question in question_set:
    question_map[question] = cnt_question
    cnt_question += 1

for concept in concept_set:
    concept_map[concept] = cnt_concept
    cnt_concept += 1

Remap student_id, question_id and concept_id: 100%|██████████| 1000/1000 [00:05<00:00, 197.94it/s]


In [69]:
TotalData = []
q_matrix = np.zeros((cnt_question, cnt_concept))

for key, value in tqdm(stu_response_data.items(), desc='Construct final data'):
    if len(value) >= least_respone_num:
        for data in value:
            TotalData.append([stu_map[data[0]], question_map[data[1]], data[2]])
            for concept in q_data.loc[q_data["problem_id"] == data[1]]['skills'].iloc[0].split(','):
                q_matrix[question_map[data[1]]][concept_map[concept]] = 1

print('Final student number: {}, Final question number: {}, Final concept number: {}, Final response number: {}'.format(cnt_stu, cnt_question, cnt_concept, len(TotalData)))

Construct final data: 100%|██████████| 1000/1000 [00:05<00:00, 191.46it/s]


Final student number: 101, Final question number: 9414, Final concept number: 584, Final response number: 19155
